In [8]:
!pip install matplotlib

In [29]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [30]:
train_revSA = pd.read_csv('temu_SA_1194_score.csv')
test_revSA = pd.read_csv("temu_SA_796_score.csv") # to be cleaned later
train_revSA.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,a0397e8f-fb5f-4241-b19d-74ecf6e4602a,Fraag Eladawy,https://play-lh.googleusercontent.com/a/ACg8oc...,لا أنصح أبدا بتحميل هذا التطبيق ، والوقوع في م...,1,2701,2.57.1,2024-04-06 18:19:35,NaN,NaN,2.57.1
1,b31994a3-91b6-4844-b304-9b08d70ad29d,Khalid_0 O,https://play-lh.googleusercontent.com/a/ACg8oc...,سهل الاستخدام أسعار وعروض مناسبة جدا وكذلك الح...,5,228,2.62.1,2024-04-30 15:48:38,NaN,NaN,2.62.1
2,bedaf0d7-51a4-4e78-820d-58190806ce94,Ľ infini,https://play-lh.googleusercontent.com/a-/ALV-U...,التطبيق رائع فيه مجموعة من المميزات الجيدة و ا...,5,13,2.63.0,2024-05-05 01:36:06,NaN,NaN,2.63.0
3,205f6ba2-4f1b-445a-99a2-30f81fb2969f,The hero,https://play-lh.googleusercontent.com/a-/ALV-U...,جدا سيء التطبيق ..لايوجد فيه خصم او كوبانات او...,1,212,2.61.0,2024-04-22 11:22:46,NaN,NaN,2.61.0
4,dd8576ff-561d-476e-90cc-037cb847679f,إدريس عبدالله,https://play-lh.googleusercontent.com/a-/ALV-U...,يمارسوا النصب بطريقه غير مباشره يقول التوصيل م...,1,33,2.63.0,2024-05-02 10:49:10,NaN,NaN,2.63.0


In [31]:
train_revSA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              1194 non-null   object
 1   userName              1194 non-null   object
 2   userImage             1194 non-null   object
 3   content               1194 non-null   object
 4   score                 1194 non-null   int64 
 5   thumbsUpCount         1194 non-null   int64 
 6   reviewCreatedVersion  1190 non-null   object
 7   at                    1194 non-null   object
 8   replyContent          48 non-null     object
 9   repliedAt             48 non-null     object
 10  appVersion            1190 non-null   object
dtypes: int64(2), object(9)
memory usage: 102.7+ KB


In [32]:
train_revSA.shape

(1194, 11)

In [33]:
train_revSA['content'].drop_duplicates(inplace = True)

### Preprocessing and Cleaning

In [36]:
# Import libraries for Establishment of NLP Pipeline 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import arabic_reshaper # for text normalization

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
train_revSA['content'][0]

'لا أنصح أبدا بتحميل هذا التطبيق ، والوقوع في مصيدة الغش ، فقد كانت تجربتي معه سيئة جدا ،  الكثير من عمليات الخداع والغش داخل التطبيق بمعنى أن الثمن المعروض على المنتج يتغير كلياً عندما تضغط عليه للشراء يزيد أكثر من النصف ولا يوجد دفع عند الإستلام كما يزعمون . كما أن جودة المنتجات تحت الصفر ، العروض كاذبة بالغالب بالأخص عروض شراء المستخدمين الجدد) أما الإعلانات خاصة التطبيق دمرت هاتفي ، انا اسمع موسيقى الإعلان تنبثق من هاتفي بدون الإعلان نفسه، لا استطيع متابعة فيديو او إرسال رساله بسببها هل هذا ي'

In [40]:
# Remove HTML tags 
cleaned_text = re.sub('<[^<]+?>', '', train_revSA['content'][0])
# Remove non-Arabic characters, digits, and punctuation
cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', cleaned_text)
# Remove extra spaces
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
cleaned_text

'لا أنصح أبدا بتحميل هذا التطبيق ، والوقوع في مصيدة الغش ، فقد كانت تجربتي معه سيئة جدا ، الكثير من عمليات الخداع والغش داخل التطبيق بمعنى أن الثمن المعروض على المنتج يتغير كلياً عندما تضغط عليه للشراء يزيد أكثر من النصف ولا يوجد دفع عند الإستلام كما يزعمون كما أن جودة المنتجات تحت الصفر ، العروض كاذبة بالغالب بالأخص عروض شراء المستخدمين الجدد أما الإعلانات خاصة التطبيق دمرت هاتفي ، انا اسمع موسيقى الإعلان تنبثق من هاتفي بدون الإعلان نفسه، لا استطيع متابعة فيديو او إرسال رساله بسببها هل هذا ي'

In [41]:
tokens = word_tokenize(cleaned_text)
print(tokens)
len(tokens)

['لا', 'أنصح', 'أبدا', 'بتحميل', 'هذا', 'التطبيق', '،', 'والوقوع', 'في', 'مصيدة', 'الغش', '،', 'فقد', 'كانت', 'تجربتي', 'معه', 'سيئة', 'جدا', '،', 'الكثير', 'من', 'عمليات', 'الخداع', 'والغش', 'داخل', 'التطبيق', 'بمعنى', 'أن', 'الثمن', 'المعروض', 'على', 'المنتج', 'يتغير', 'كلياً', 'عندما', 'تضغط', 'عليه', 'للشراء', 'يزيد', 'أكثر', 'من', 'النصف', 'ولا', 'يوجد', 'دفع', 'عند', 'الإستلام', 'كما', 'يزعمون', 'كما', 'أن', 'جودة', 'المنتجات', 'تحت', 'الصفر', '،', 'العروض', 'كاذبة', 'بالغالب', 'بالأخص', 'عروض', 'شراء', 'المستخدمين', 'الجدد', 'أما', 'الإعلانات', 'خاصة', 'التطبيق', 'دمرت', 'هاتفي', '،', 'انا', 'اسمع', 'موسيقى', 'الإعلان', 'تنبثق', 'من', 'هاتفي', 'بدون', 'الإعلان', 'نفسه،', 'لا', 'استطيع', 'متابعة', 'فيديو', 'او', 'إرسال', 'رساله', 'بسببها', 'هل', 'هذا', 'ي']


92

In [42]:
print("Arabic stop-words : ", set(stopwords.words('arabic')))

Arabic stop-words :  {'هَذَيْنِ', 'نَّ', 'ليت', 'أهلا', 'قاطبة', 'بها', 'هاء', 'إياهم', 'بات', 'طَق', 'سبت', 'لست', 'ليسوا', 'متى', 'تسعة', 'إلا', 'سقى', 'سرا', 'لعلَّ', 'كى', 'هبّ', 'خمسمئة', 'أُفٍّ', 'بي', 'سمعا', 'تموز', 'فو', 'قبل', 'بلى', 'أقل', 'هناك', 'مايو', 'أمّا', 'ذ', 'منذ', 'ضحوة', 'أنًّ', 'ذلكم', 'هيّا', 'درهم', 'هذا', 'علم', 'كانون', 'يفعلان', 'رأى', 'نوفمبر', 'رجع', 'بغتة', 'ألفى', 'ثلاثون', 'مثل', 'حمو', 'كرب', 'كأنّ', 'لسنا', 'أربعاء', 'باء', 'إلى', 'ب', 'لهم', 'خميس', 'ما', 'شين', 'ي', 'كليهما', 'طاء', 'تعسا', 'ذوا', 'ه', 'ثامن', 'الذي', 'كأن', 'كذا', 'لكي', 'ن', 'إياي', 'ّأيّان', 'حادي', 'تفعلون', 'هَذا', 'يناير', 'مما', 'أيا', 'ضاد', 'دون', 'ظاء', 'هما', 'آي', 'سبعون', 'أنبأ', 'حين', 'اللاتي', 'سبعة', 'إليكنّ', 'لكنما', 'ألف', 'ليست', 'أيلول', 'حيث', 'اللواتي', 'إياكم', 'لسن', 'ثلاثة', 'فمن', 'أفعل به', 'بما', 'أيضا', 'بهن', 'واهاً', 'و', 'ذواتي', 'كليكما', 'لدى', 'عما', 'إيه', 'منه', 'بين', 'إياكن', 'خبَّر', 'يورو', 'هنا', 'هَاتِي', 'ليستا', 'أخٌ', 'ذه', 'وإذ', 'ول

In [43]:
stop_words = set(stopwords.words('arabic'))

# text cleaning function
def clean_text(text):
    # Remove HTML tags 
    cleaned_text = re.sub('<[^<]+?>', '', text)
    # Remove non-Arabic characters, digits, and punctuation
    cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', cleaned_text)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

# text normalization function
def normalize_text(text):
    # Normalize Arabic characters
    normalized_text = arabic_reshaper.reshape(text)
    return normalized_text

In [44]:
content_tokens = []

# Process each text
for i in range(len(train_revSA)):
    # Get Arabic text from MongoDB document
    arabic_text = train_revSA['content'][i]
    # print("before : ", arabic_text)
    
    # Clean the text
    cleaned_text = clean_text(arabic_text)
    
    # Tokenize the cleaned text
    tokens = word_tokenize(cleaned_text)
    
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Normalize the text
    normalized_text = normalize_text(' '.join(filtered_tokens))
    
    # Print or store the processed text
    content_tokens.append(normalized_text)
    # print("after : ", normalized_text)
    # print("*"*50)

train_revSA['content_tokens'] = content_tokens

In [45]:
train_revSA[['content', 'content_tokens']].head(10)

,content,content_tokens
0,لا أنصح أبدا بتحميل هذا التطبيق ، والوقوع في م...,ﺃﻧﺼﺢ ﺑﺘﺤﻤﻴﻞ ﺍﻟﺘﻄﺒﻴﻖ ، ﻭﺍﻟﻮﻗﻮﻉ ﻣﺼﻴﺪﺓ ﺍﻟﻐﺶ ، ﻓﻘﺪ...
1,سهل الاستخدام أسعار وعروض مناسبة جدا وكذلك الح...,ﺳﻬﻞ ﺍﻻﺳﺘﺨﺪﺍﻡ ﺃﺳﻌﺎﺭ ﻭﻋﺮﻭﺽ ﻣﻨﺎﺳﺒﺔ ﺟﺪﺍ ﻭﻛﺬﻟﻚ ﺍﻟﺤﺼ...
2,التطبيق رائع فيه مجموعة من المميزات الجيدة و ا...,ﺍﻟﺘﻄﺒﻴﻖ ﺭﺍﺋﻊ ﻣﺠﻤﻮﻋﺔ ﺍﻟﻤﻤﻴﺰﺍﺕ ﺍﻟﺠﻴﺪﺓ ﺍﻷﺛﻤﻨﺔ ﺟﺪ ...
3,جدا سيء التطبيق ..لايوجد فيه خصم او كوبانات او...,ﺟﺪﺍ ﺳﻲﺀ ﺍﻟﺘﻄﺒﻴﻖ ﻻﻳﻮﺟﺪ ﺧﺼﻢ ﺍﻭ ﻛﻮﺑﺎﻧﺎﺕ ﺍﻭ ﺍﻭ ﺍﻧﻲ...
4,يمارسوا النصب بطريقه غير مباشره يقول التوصيل م...,ﻳﻤﺎﺭﺳﻮﺍ ﺍﻟﻨﺼﺐ ﺑﻄﺮﻳﻘﻪ ﻣﺒﺎﺷﺮﻩ ﻳﻘﻮﻝ ﺍﻟﺘﻮﺻﻴﻞ ﻣﺠﺎﻧﺎ...
5,على حسب تجربتي افضل اسعار هناء توصيل مجاني وعر...,ﺗﺠﺮﺑﺘﻲ ﺍﻓﻀﻞ ﺍﺳﻌﺎﺭ ﻫﻨﺎﺀ ﺗﻮﺻﻴﻞ ﻣﺠﺎﻧﻲ ﻭﻋﺮﻭﺽ ﻣﻤﺘﺎﺯ...
6,ممل جدا من ناحية الدعايات الزائدة .. و موضوع ا...,ﻣﻤﻞ ﺟﺪﺍ ﻧﺎﺣﻴﺔ ﺍﻟﺪﻋﺎﻳﺎﺕ ﺍﻟﺰﺍﺋﺪﺓ ﻣﻮﺿﻮﻉ ﺍﻻﺷﺘﺮﺍﻙ ﺍ...
7,المنتوجات الموجوده في التطبيق جودتها ثلاث نجوم...,ﺍﻟﻤﻨﺘﻮﺟﺎﺕ ﺍﻟﻤﻮﺟﻮﺩﻩ ﺍﻟﺘﻄﺒﻴﻖ ﺟﻮﺩﺗﻬﺎ ﻧﺠﻮﻡ ﻭﺍﻧﺎ ﻋﻤ...
8,اداة البحث لا تنفع. فعندما تبحث عن شيء معين يظ...,ﺍﺩﺍﺓ ﺍﻟﺒﺤﺚ ﺗﻨﻔﻊ ﻓﻌﻨﺪﻣﺎ ﺗﺒﺤﺚ ﺷﻲﺀ ﻣﻌﻴﻦ ﻳﻈﻬﺮ ﺍﻟﺒﺤ...
9,لفترة كبيرة من الزمن كنت متردد في تنزيل التطبي...,ﻟﻔﺘﺮﺓ ﻛﺒﻴﺮﺓ ﺍﻟﺰﻣﻦ ﻛﻨﺖ ﻣﺘﺮﺩﺩ ﺗﻨﺰﻳﻞ ﺍﻟﺘﻄﺒﻴﻖ ﻛﻨﺖ ...


## Stemming and Lemmatization

In [46]:
from nltk.stem import WordNetLemmatizer, ISRIStemmer
from nltk.tokenize import word_tokenize
import nltk

# Arabic stemmer
stemmer = ISRIStemmer()

# WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

content_lemmatized = []

for i in range(len(train_revSA)):

    arabic_text = train_revSA['content_tokens'][i]

    # Tokenize the sentence
    tokens = word_tokenize(arabic_text)

    # Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(token) for token in tokens]

    # Stemming
    stemmed_words = [stemmer.stem(token) for token in tokens]

    # store the lemmatized words because Lemmatization return the base or dictionary form of a word,
    # known as the lemma, by considering the context and part of speech of the word.
    content_lemmatized.append(" ".join(lemmatized_words))

    if i == 1 :
        print("arabic_text : ", arabic_text)
        print("-"*50)
        print("Lemmatized words:", lemmatized_words)
        print("-"*50)
        print("Stemmed words:", stemmed_words)


train_revSA["content_lemmatized"] = content_lemmatized

arabic_text :  ﺳﻬﻞ ﺍﻻﺳﺘﺨﺪﺍﻡ ﺃﺳﻌﺎﺭ ﻭﻋﺮﻭﺽ ﻣﻨﺎﺳﺒﺔ ﺟﺪﺍ ﻭﻛﺬﻟﻚ ﺍﻟﺤﺼﻮﻝ ﺧﺼﻮﻣﺎﺕ ﺍﻟﺸﺮﺍﺀ ﺳﺮﻋﺔ ﺍﻹﻧﺠﺎﺯ ﺑﺘﻮﺻﻴﻞ ﻃﻠﺒﻚ ﺍﻟﺘﺎﺭﻳﺦ ﺍﻟﻤﺤﺪﺩ ﻻﺳﺘﻼﻡ ﺷﺤﻨﺘﻚ ﻭﺇﻣﻜﺎﻧﻴﺎﺕ ﻛﺜﻴﺮﻩ ﺍﺳﺘﺮﺟﺎﻉ ﺍﻟﻤﻨﺘﺠﺎﺕ ﺍﻟﻠﻲ ﻣﺎﺗﻌﻤﻞ ﺍﻭ ﺗﺎﻟﻒ ﺍﻭ ﺷﺮﺍﺀ ﺑﺎﻟﻐﻠﻂ ﻭﻓﻲ ﺣﺎﻟﺔ ﺗﻌﺜﺮﺕ ﻭﻟﻢ ﺗﺘﻤﻜﻦ ﺍﻟﻤﻌﺮﻓﺔ ﺑﺎﻣﻜﺎﻧﻚ ﺍﻟﺘﻮﺍﺻﻞ ﺑﻔﺮﻳﻖ ﺍﻟﺪﻋﻢ ﻭﺍﺧﻴﺮﺁ ﺍﻷﻓﻀﻞ ﺍﻹﻃﻼﻕ ﺍﻧﻪ ﻟﻜﻞ ﻣﺸﺘﺮﻙ ﺣﺴﺎﺏ ﺗﺄﻣﻴﻦ ﻟﺪﻳﻬﻢ ﺣﺎﻟﺔ ﺍﺳﺘﺮﺟﺎﻉ ﺍﻱ ﻣﻨﺘﺞ ﺍﻟﻤﺒﻠﻎ ﻳﻮﺩﻉ ﻟﺤﺴﺎﺑﻚ ﺍﻟﺘﺎﻣﻴﻨﻲ، ﻭﻫﻨﺎﻙ ﺧﺎﺻﻴﺔ ﺗﻌﺪﻳﻞ ﺍﻻﺳﻌﺎﺭ ﺍﻟﺸﺮﺍﺀ ﻭﺗﺴﺘﻔﻴﺪ ﺍﻟﻤﺒﺎﻟﻎ ﻋﻨﺪﺍﻟﺸﺮﺍﺀ ﻣﺮﻩ ﺍﺧﺮﻯ ﺩﻣﺘﻢ ﺑﺨﻴﺮ
--------------------------------------------------
Lemmatized words: ['ﺳﻬﻞ', 'ﺍﻻﺳﺘﺨﺪﺍﻡ', 'ﺃﺳﻌﺎﺭ', 'ﻭﻋﺮﻭﺽ', 'ﻣﻨﺎﺳﺒﺔ', 'ﺟﺪﺍ', 'ﻭﻛﺬﻟﻚ', 'ﺍﻟﺤﺼﻮﻝ', 'ﺧﺼﻮﻣﺎﺕ', 'ﺍﻟﺸﺮﺍﺀ', 'ﺳﺮﻋﺔ', 'ﺍﻹﻧﺠﺎﺯ', 'ﺑﺘﻮﺻﻴﻞ', 'ﻃﻠﺒﻚ', 'ﺍﻟﺘﺎﺭﻳﺦ', 'ﺍﻟﻤﺤﺪﺩ', 'ﻻﺳﺘﻼﻡ', 'ﺷﺤﻨﺘﻚ', 'ﻭﺇﻣﻜﺎﻧﻴﺎﺕ', 'ﻛﺜﻴﺮﻩ', 'ﺍﺳﺘﺮﺟﺎﻉ', 'ﺍﻟﻤﻨﺘﺠﺎﺕ', 'ﺍﻟﻠﻲ', 'ﻣﺎﺗﻌﻤﻞ', 'ﺍﻭ', 'ﺗﺎﻟﻒ', 'ﺍﻭ', 'ﺷﺮﺍﺀ', 'ﺑﺎﻟﻐﻠﻂ', 'ﻭﻓﻲ', 'ﺣﺎﻟﺔ', 'ﺗﻌﺜﺮﺕ', 'ﻭﻟﻢ', 'ﺗﺘﻤﻜﻦ', 'ﺍﻟﻤﻌﺮﻓﺔ', 'ﺑﺎﻣﻜﺎﻧﻚ', 'ﺍﻟﺘﻮﺍﺻﻞ', 'ﺑﻔﺮﻳﻖ', 'ﺍﻟﺪﻋﻢ', 'ﻭﺍﺧﻴﺮﺁ', 'ﺍﻷﻓﻀﻞ', 'ﺍﻹﻃﻼﻕ', 'ﺍﻧﻪ', 'ﻟﻜﻞ', 'ﻣﺸﺘﺮﻙ', 'ﺣﺴﺎﺏ', 'ﺗﺄﻣﻴﻦ', 'ﻟﺪﻳﻬﻢ', 'ﺣﺎﻟﺔ', 'ﺍﺳﺘﺮﺟﺎﻉ', 'ﺍﻱ', 'ﻣﻨﺘﺞ', 'ﺍﻟﻤﺒﻠﻎ', 'ﻳﻮﺩﻉ', 'ﻟﺤﺴﺎﺑﻚ

In [62]:
train_revSA[['content', 'content_tokens', 'content_lemmatized']].head(10)

,content,content_tokens,content_lemmatized
0,لا أنصح أبدا بتحميل هذا التطبيق ، والوقوع في م...,ﺃﻧﺼﺢ ﺑﺘﺤﻤﻴﻞ ﺍﻟﺘﻄﺒﻴﻖ ، ﻭﺍﻟﻮﻗﻮﻉ ﻣﺼﻴﺪﺓ ﺍﻟﻐﺶ ، ﻓﻘﺪ...,ﺃﻧﺼﺢ ﺑﺘﺤﻤﻴﻞ ﺍﻟﺘﻄﺒﻴﻖ ، ﻭﺍﻟﻮﻗﻮﻉ ﻣﺼﻴﺪﺓ ﺍﻟﻐﺶ ، ﻓﻘﺪ...
1,سهل الاستخدام أسعار وعروض مناسبة جدا وكذلك الح...,ﺳﻬﻞ ﺍﻻﺳﺘﺨﺪﺍﻡ ﺃﺳﻌﺎﺭ ﻭﻋﺮﻭﺽ ﻣﻨﺎﺳﺒﺔ ﺟﺪﺍ ﻭﻛﺬﻟﻚ ﺍﻟﺤﺼ...,ﺳﻬﻞ ﺍﻻﺳﺘﺨﺪﺍﻡ ﺃﺳﻌﺎﺭ ﻭﻋﺮﻭﺽ ﻣﻨﺎﺳﺒﺔ ﺟﺪﺍ ﻭﻛﺬﻟﻚ ﺍﻟﺤﺼ...
2,التطبيق رائع فيه مجموعة من المميزات الجيدة و ا...,ﺍﻟﺘﻄﺒﻴﻖ ﺭﺍﺋﻊ ﻣﺠﻤﻮﻋﺔ ﺍﻟﻤﻤﻴﺰﺍﺕ ﺍﻟﺠﻴﺪﺓ ﺍﻷﺛﻤﻨﺔ ﺟﺪ ...,ﺍﻟﺘﻄﺒﻴﻖ ﺭﺍﺋﻊ ﻣﺠﻤﻮﻋﺔ ﺍﻟﻤﻤﻴﺰﺍﺕ ﺍﻟﺠﻴﺪﺓ ﺍﻷﺛﻤﻨﺔ ﺟﺪ ...
3,جدا سيء التطبيق ..لايوجد فيه خصم او كوبانات او...,ﺟﺪﺍ ﺳﻲﺀ ﺍﻟﺘﻄﺒﻴﻖ ﻻﻳﻮﺟﺪ ﺧﺼﻢ ﺍﻭ ﻛﻮﺑﺎﻧﺎﺕ ﺍﻭ ﺍﻭ ﺍﻧﻲ...,ﺟﺪﺍ ﺳﻲﺀ ﺍﻟﺘﻄﺒﻴﻖ ﻻﻳﻮﺟﺪ ﺧﺼﻢ ﺍﻭ ﻛﻮﺑﺎﻧﺎﺕ ﺍﻭ ﺍﻭ ﺍﻧﻲ...
4,يمارسوا النصب بطريقه غير مباشره يقول التوصيل م...,ﻳﻤﺎﺭﺳﻮﺍ ﺍﻟﻨﺼﺐ ﺑﻄﺮﻳﻘﻪ ﻣﺒﺎﺷﺮﻩ ﻳﻘﻮﻝ ﺍﻟﺘﻮﺻﻴﻞ ﻣﺠﺎﻧﺎ...,ﻳﻤﺎﺭﺳﻮﺍ ﺍﻟﻨﺼﺐ ﺑﻄﺮﻳﻘﻪ ﻣﺒﺎﺷﺮﻩ ﻳﻘﻮﻝ ﺍﻟﺘﻮﺻﻴﻞ ﻣﺠﺎﻧﺎ...
5,على حسب تجربتي افضل اسعار هناء توصيل مجاني وعر...,ﺗﺠﺮﺑﺘﻲ ﺍﻓﻀﻞ ﺍﺳﻌﺎﺭ ﻫﻨﺎﺀ ﺗﻮﺻﻴﻞ ﻣﺠﺎﻧﻲ ﻭﻋﺮﻭﺽ ﻣﻤﺘﺎﺯ...,ﺗﺠﺮﺑﺘﻲ ﺍﻓﻀﻞ ﺍﺳﻌﺎﺭ ﻫﻨﺎﺀ ﺗﻮﺻﻴﻞ ﻣﺠﺎﻧﻲ ﻭﻋﺮﻭﺽ ﻣﻤﺘﺎﺯ...
6,ممل جدا من ناحية الدعايات الزائدة .. و موضوع ا...,ﻣﻤﻞ ﺟﺪﺍ ﻧﺎﺣﻴﺔ ﺍﻟﺪﻋﺎﻳﺎﺕ ﺍﻟﺰﺍﺋﺪﺓ ﻣﻮﺿﻮﻉ ﺍﻻﺷﺘﺮﺍﻙ ﺍ...,ﻣﻤﻞ ﺟﺪﺍ ﻧﺎﺣﻴﺔ ﺍﻟﺪﻋﺎﻳﺎﺕ ﺍﻟﺰﺍﺋﺪﺓ ﻣﻮﺿﻮﻉ ﺍﻻﺷﺘﺮﺍﻙ ﺍ...
7,المنتوجات الموجوده في التطبيق جودتها ثلاث نجوم...,ﺍﻟﻤﻨﺘﻮﺟﺎﺕ ﺍﻟﻤﻮﺟﻮﺩﻩ ﺍﻟﺘﻄﺒﻴﻖ ﺟﻮﺩﺗﻬﺎ ﻧﺠﻮﻡ ﻭﺍﻧﺎ ﻋﻤ...,ﺍﻟﻤﻨﺘﻮﺟﺎﺕ ﺍﻟﻤﻮﺟﻮﺩﻩ ﺍﻟﺘﻄﺒﻴﻖ ﺟﻮﺩﺗﻬﺎ ﻧﺠﻮﻡ ﻭﺍﻧﺎ ﻋﻤ...
8,اداة البحث لا تنفع. فعندما تبحث عن شيء معين يظ...,ﺍﺩﺍﺓ ﺍﻟﺒﺤﺚ ﺗﻨﻔﻊ ﻓﻌﻨﺪﻣﺎ ﺗﺒﺤﺚ ﺷﻲﺀ ﻣﻌﻴﻦ ﻳﻈﻬﺮ ﺍﻟﺒﺤ...,ﺍﺩﺍﺓ ﺍﻟﺒﺤﺚ ﺗﻨﻔﻊ ﻓﻌﻨﺪﻣﺎ ﺗﺒﺤﺚ ﺷﻲﺀ ﻣﻌﻴﻦ ﻳﻈﻬﺮ ﺍﻟﺒﺤ...
9,لفترة كبيرة من الزمن كنت متردد في تنزيل التطبي...,ﻟﻔﺘﺮﺓ ﻛﺒﻴﺮﺓ ﺍﻟﺰﻣﻦ ﻛﻨﺖ ﻣﺘﺮﺩﺩ ﺗﻨﺰﻳﻞ ﺍﻟﺘﻄﺒﻴﻖ ﻛﻨﺖ ...,ﻟﻔﺘﺮﺓ ﻛﺒﻴﺮﺓ ﺍﻟﺰﻣﻦ ﻛﻨﺖ ﻣﺘﺮﺩﺩ ﺗﻨﺰﻳﻞ ﺍﻟﺘﻄﺒﻴﻖ ﻛﻨﺖ ...


In [74]:
from tashaphyne.stemming import ArabicLightStemmer
from nltk.tokenize import word_tokenize
import pyarabic.araby as araby


ArListem = ArabicLightStemmer()
content_Stemitized = []

for i in range(len(train_revSA)):
    arabic_text = train_revSA['content_tokens'][i]
    tokens = word_tokenize(arabic_text)
    stemmed_words = [ArListem.light_stem(token) for token in tokens]
    tagged_words = ArListem.get_stem()  
    print(tagged_words)
    content_Stemitized.append(" ".join(stemmed_words))
    
    if i == 1 :
        print("stemmed_words : ", stemmed_words)
        print("-"*50)
        print("tagged_words :", tagged_words)

train_revSA["content_stemmatized"] = content_Stemitized

ﺑﺴﺒﺒﻬﺎ
ﺑﺨﻴﺮ
stemmed_words :  ['ﺳﻬﻞ', 'ﺍﻻﺳﺘﺨﺪﺍﻡ', 'ﺃﺳﻌﺎﺭ', 'ﻭﻋﺮﻭﺽ', 'ﻣﻨﺎﺳﺒﺔ', 'ﺟﺪﺍ', 'ﻭﻛﺬﻟﻚ', 'ﺍﻟﺤﺼﻮﻝ', 'ﺧﺼﻮﻣﺎﺕ', 'ﺍﻟﺸﺮﺍﺀ', 'ﺳﺮﻋﺔ', 'ﺍﻹﻧﺠﺎﺯ', 'ﺑﺘﻮﺻﻴﻞ', 'ﻃﻠﺒﻚ', 'ﺍﻟﺘﺎﺭﻳﺦ', 'ﺍﻟﻤﺤﺪﺩ', 'ﻻﺳﺘﻼﻡ', 'ﺷﺤﻨﺘﻚ', 'ﻭﺇﻣﻜﺎﻧﻴﺎﺕ', 'ﻛﺜﻴﺮﻩ', 'ﺍﺳﺘﺮﺟﺎﻉ', 'ﺍﻟﻤﻨﺘﺠﺎﺕ', 'ﺍﻟﻠﻲ', 'ﻣﺎﺗﻌﻤﻞ', 'ﺍﻭ', 'ﺗﺎﻟﻒ', 'ﺍﻭ', 'ﺷﺮﺍﺀ', 'ﺑﺎﻟﻐﻠﻂ', 'ﻭﻓﻲ', 'ﺣﺎﻟﺔ', 'ﺗﻌﺜﺮﺕ', 'ﻭﻟﻢ', 'ﺗﺘﻤﻜﻦ', 'ﺍﻟﻤﻌﺮﻓﺔ', 'ﺑﺎﻣﻜﺎﻧﻚ', 'ﺍﻟﺘﻮﺍﺻﻞ', 'ﺑﻔﺮﻳﻖ', 'ﺍﻟﺪﻋﻢ', 'ﻭﺍﺧﻴﺮﺁ', 'ﺍﻷﻓﻀﻞ', 'ﺍﻹﻃﻼﻕ', 'ﺍﻧﻪ', 'ﻟﻜﻞ', 'ﻣﺸﺘﺮﻙ', 'ﺣﺴﺎﺏ', 'ﺗﺄﻣﻴﻦ', 'ﻟﺪﻳﻬﻢ', 'ﺣﺎﻟﺔ', 'ﺍﺳﺘﺮﺟﺎﻉ', 'ﺍﻱ', 'ﻣﻨﺘﺞ', 'ﺍﻟﻤﺒﻠﻎ', 'ﻳﻮﺩﻉ', 'ﻟﺤﺴﺎﺑﻚ', 'ﺍﻟﺘﺎﻣﻴﻨﻲ،', 'ﻭﻫﻨﺎﻙ', 'ﺧﺎﺻﻴﺔ', 'ﺗﻌﺪﻳﻞ', 'ﺍﻻﺳﻌﺎﺭ', 'ﺍﻟﺸﺮﺍﺀ', 'ﻭﺗﺴﺘﻔﻴﺪ', 'ﺍﻟﻤﺒﺎﻟﻎ', 'ﻋﻨﺪﺍﻟﺸﺮﺍﺀ', 'ﻣﺮﻩ', 'ﺍﺧﺮﻯ', 'ﺩﻣﺘﻢ', 'ﺑﺨﻴﺮ']
--------------------------------------------------
tagged_words : ﺑﺨﻴﺮ
ﺃﺳﺘﻤﺘﻊ
ﺍﻟﺰﺑﺎﺋﻦ
ﻣﺠﺎﻧﺎ،
ﺍﻟﻌﺎﻟﻢ
ﻧﺠﻤﺔ
ﻋﻠﻲ
ﺍﻟﺤﺬﻑ
ﺍﻟﻤﺼﺪﺍﻗﻴﺔ
ﺍﻷﻣﺎﻥ
ﺃﺳﻌﺎﺭﺍ
ﺍﻟﻘﻠﻴﻞ
ﺍﻟﻌﻤﻼﺀ
ﺍﺑﺪﺍ
ﺍﻟﺘﻄﺒﻴﻖ
ﻣﻌﻪ
ﺟﺪﺍ
ﺑﺴﺮﻋﻪ
ﻛﺜﻴﺮﺍ
ﻟﻠﺘﻄﺒﻴﻖ
ﻃﺒﻌﺎ
ﺷﻜﺮ
ﻟﻼﺳﻒ
ﺳﻴﺌﺔ
ﻧﻬﺎﺋﻴﺎ
ﺍﻟﺼﻮﺭﻩ
ﻟﻠﺠﺪﺩ
ﺑﺘﺤﻤﻴﻠﺔ
ﻛﺒﻴﺮﺍ
ﻭﺍﻻﺣﺘﻴﺎﻝ
ﻓﺎﺋﺪﻩ
ﺍﻛﺴﺒﺮﺱ
ﺍﻻﺳﺘﻼﻡ
ﺍﻟﻘﻴﺎﺱ
ﻧﻬﺎﺋﻴﺎ
ﺍﻟﻤﻀﻠﻞ
ﺗﺤﻴﺎﺗﻲ
ﻟﻼﻋﺪﺍﺩﺍﺕ
ﺍﻭ
ﺍﻟﻤﺘﺴﻮﻕ
ﺷﻜﺮﺍ
ﺍﻟﻨﻬﺞ
ﻟﻔﻌﻠﺖ
ﺍﺷﺘﺮﻳﺘ

In [75]:
train_revSA[['content', 'content_tokens', 'content_lemmatized','content_stemmatized']].head(10)

,content,content_tokens,content_lemmatized,content_stemmatized
0,لا أنصح أبدا بتحميل هذا التطبيق ، والوقوع في م...,ﺃﻧﺼﺢ ﺑﺘﺤﻤﻴﻞ ﺍﻟﺘﻄﺒﻴﻖ ، ﻭﺍﻟﻮﻗﻮﻉ ﻣﺼﻴﺪﺓ ﺍﻟﻐﺶ ، ﻓﻘﺪ...,ﺃﻧﺼﺢ ﺑﺘﺤﻤﻴﻞ ﺍﻟﺘﻄﺒﻴﻖ ، ﻭﺍﻟﻮﻗﻮﻉ ﻣﺼﻴﺪﺓ ﺍﻟﻐﺶ ، ﻓﻘﺪ...,ﺃﻧﺼﺢ ﺑﺘﺤﻤﻴﻞ ﺍﻟﺘﻄﺒﻴﻖ ، ﻭﺍﻟﻮﻗﻮﻉ ﻣﺼﻴﺪﺓ ﺍﻟﻐﺶ ، ﻓﻘﺪ...
1,سهل الاستخدام أسعار وعروض مناسبة جدا وكذلك الح...,ﺳﻬﻞ ﺍﻻﺳﺘﺨﺪﺍﻡ ﺃﺳﻌﺎﺭ ﻭﻋﺮﻭﺽ ﻣﻨﺎﺳﺒﺔ ﺟﺪﺍ ﻭﻛﺬﻟﻚ ﺍﻟﺤﺼ...,ﺳﻬﻞ ﺍﻻﺳﺘﺨﺪﺍﻡ ﺃﺳﻌﺎﺭ ﻭﻋﺮﻭﺽ ﻣﻨﺎﺳﺒﺔ ﺟﺪﺍ ﻭﻛﺬﻟﻚ ﺍﻟﺤﺼ...,ﺳﻬﻞ ﺍﻻﺳﺘﺨﺪﺍﻡ ﺃﺳﻌﺎﺭ ﻭﻋﺮﻭﺽ ﻣﻨﺎﺳﺒﺔ ﺟﺪﺍ ﻭﻛﺬﻟﻚ ﺍﻟﺤﺼ...
2,التطبيق رائع فيه مجموعة من المميزات الجيدة و ا...,ﺍﻟﺘﻄﺒﻴﻖ ﺭﺍﺋﻊ ﻣﺠﻤﻮﻋﺔ ﺍﻟﻤﻤﻴﺰﺍﺕ ﺍﻟﺠﻴﺪﺓ ﺍﻷﺛﻤﻨﺔ ﺟﺪ ...,ﺍﻟﺘﻄﺒﻴﻖ ﺭﺍﺋﻊ ﻣﺠﻤﻮﻋﺔ ﺍﻟﻤﻤﻴﺰﺍﺕ ﺍﻟﺠﻴﺪﺓ ﺍﻷﺛﻤﻨﺔ ﺟﺪ ...,ﺍﻟﺘﻄﺒﻴﻖ ﺭﺍﺋﻊ ﻣﺠﻤﻮﻋﺔ ﺍﻟﻤﻤﻴﺰﺍﺕ ﺍﻟﺠﻴﺪﺓ ﺍﻷﺛﻤﻨﺔ ﺟﺪ ...
3,جدا سيء التطبيق ..لايوجد فيه خصم او كوبانات او...,ﺟﺪﺍ ﺳﻲﺀ ﺍﻟﺘﻄﺒﻴﻖ ﻻﻳﻮﺟﺪ ﺧﺼﻢ ﺍﻭ ﻛﻮﺑﺎﻧﺎﺕ ﺍﻭ ﺍﻭ ﺍﻧﻲ...,ﺟﺪﺍ ﺳﻲﺀ ﺍﻟﺘﻄﺒﻴﻖ ﻻﻳﻮﺟﺪ ﺧﺼﻢ ﺍﻭ ﻛﻮﺑﺎﻧﺎﺕ ﺍﻭ ﺍﻭ ﺍﻧﻲ...,ﺟﺪﺍ ﺳﻲﺀ ﺍﻟﺘﻄﺒﻴﻖ ﻻﻳﻮﺟﺪ ﺧﺼﻢ ﺍﻭ ﻛﻮﺑﺎﻧﺎﺕ ﺍﻭ ﺍﻭ ﺍﻧﻲ...
4,يمارسوا النصب بطريقه غير مباشره يقول التوصيل م...,ﻳﻤﺎﺭﺳﻮﺍ ﺍﻟﻨﺼﺐ ﺑﻄﺮﻳﻘﻪ ﻣﺒﺎﺷﺮﻩ ﻳﻘﻮﻝ ﺍﻟﺘﻮﺻﻴﻞ ﻣﺠﺎﻧﺎ...,ﻳﻤﺎﺭﺳﻮﺍ ﺍﻟﻨﺼﺐ ﺑﻄﺮﻳﻘﻪ ﻣﺒﺎﺷﺮﻩ ﻳﻘﻮﻝ ﺍﻟﺘﻮﺻﻴﻞ ﻣﺠﺎﻧﺎ...,ﻳﻤﺎﺭﺳﻮﺍ ﺍﻟﻨﺼﺐ ﺑﻄﺮﻳﻘﻪ ﻣﺒﺎﺷﺮﻩ ﻳﻘﻮﻝ ﺍﻟﺘﻮﺻﻴﻞ ﻣﺠﺎﻧﺎ...
5,على حسب تجربتي افضل اسعار هناء توصيل مجاني وعر...,ﺗﺠﺮﺑﺘﻲ ﺍﻓﻀﻞ ﺍﺳﻌﺎﺭ ﻫﻨﺎﺀ ﺗﻮﺻﻴﻞ ﻣﺠﺎﻧﻲ ﻭﻋﺮﻭﺽ ﻣﻤﺘﺎﺯ...,ﺗﺠﺮﺑﺘﻲ ﺍﻓﻀﻞ ﺍﺳﻌﺎﺭ ﻫﻨﺎﺀ ﺗﻮﺻﻴﻞ ﻣﺠﺎﻧﻲ ﻭﻋﺮﻭﺽ ﻣﻤﺘﺎﺯ...,ﺗﺠﺮﺑﺘﻲ ﺍﻓﻀﻞ ﺍﺳﻌﺎﺭ ﻫﻨﺎﺀ ﺗﻮﺻﻴﻞ ﻣﺠﺎﻧﻲ ﻭﻋﺮﻭﺽ ﻣﻤﺘﺎﺯ...
6,ممل جدا من ناحية الدعايات الزائدة .. و موضوع ا...,ﻣﻤﻞ ﺟﺪﺍ ﻧﺎﺣﻴﺔ ﺍﻟﺪﻋﺎﻳﺎﺕ ﺍﻟﺰﺍﺋﺪﺓ ﻣﻮﺿﻮﻉ ﺍﻻﺷﺘﺮﺍﻙ ﺍ...,ﻣﻤﻞ ﺟﺪﺍ ﻧﺎﺣﻴﺔ ﺍﻟﺪﻋﺎﻳﺎﺕ ﺍﻟﺰﺍﺋﺪﺓ ﻣﻮﺿﻮﻉ ﺍﻻﺷﺘﺮﺍﻙ ﺍ...,ﻣﻤﻞ ﺟﺪﺍ ﻧﺎﺣﻴﺔ ﺍﻟﺪﻋﺎﻳﺎﺕ ﺍﻟﺰﺍﺋﺪﺓ ﻣﻮﺿﻮﻉ ﺍﻻﺷﺘﺮﺍﻙ ﺍ...
7,المنتوجات الموجوده في التطبيق جودتها ثلاث نجوم...,ﺍﻟﻤﻨﺘﻮﺟﺎﺕ ﺍﻟﻤﻮﺟﻮﺩﻩ ﺍﻟﺘﻄﺒﻴﻖ ﺟﻮﺩﺗﻬﺎ ﻧﺠﻮﻡ ﻭﺍﻧﺎ ﻋﻤ...,ﺍﻟﻤﻨﺘﻮﺟﺎﺕ ﺍﻟﻤﻮﺟﻮﺩﻩ ﺍﻟﺘﻄﺒﻴﻖ ﺟﻮﺩﺗﻬﺎ ﻧﺠﻮﻡ ﻭﺍﻧﺎ ﻋﻤ...,ﺍﻟﻤﻨﺘﻮﺟﺎﺕ ﺍﻟﻤﻮﺟﻮﺩﻩ ﺍﻟﺘﻄﺒﻴﻖ ﺟﻮﺩﺗﻬﺎ ﻧﺠﻮﻡ ﻭﺍﻧﺎ ﻋﻤ...
8,اداة البحث لا تنفع. فعندما تبحث عن شيء معين يظ...,ﺍﺩﺍﺓ ﺍﻟﺒﺤﺚ ﺗﻨﻔﻊ ﻓﻌﻨﺪﻣﺎ ﺗﺒﺤﺚ ﺷﻲﺀ ﻣﻌﻴﻦ ﻳﻈﻬﺮ ﺍﻟﺒﺤ...,ﺍﺩﺍﺓ ﺍﻟﺒﺤﺚ ﺗﻨﻔﻊ ﻓﻌﻨﺪﻣﺎ ﺗﺒﺤﺚ ﺷﻲﺀ ﻣﻌﻴﻦ ﻳﻈﻬﺮ ﺍﻟﺒﺤ...,ﺍﺩﺍﺓ ﺍﻟﺒﺤﺚ ﺗﻨﻔﻊ ﻓﻌﻨﺪﻣﺎ ﺗﺒﺤﺚ ﺷﻲﺀ ﻣﻌﻴﻦ ﻳﻈﻬﺮ ﺍﻟﺒﺤ...
9,لفترة كبيرة من الزمن كنت متردد في تنزيل التطبي...,ﻟﻔﺘﺮﺓ ﻛﺒﻴﺮﺓ ﺍﻟﺰﻣﻦ ﻛﻨﺖ ﻣﺘﺮﺩﺩ ﺗﻨﺰﻳﻞ ﺍﻟﺘﻄﺒﻴﻖ ﻛﻨﺖ ...,ﻟﻔﺘﺮﺓ ﻛﺒﻴﺮﺓ ﺍﻟﺰﻣﻦ ﻛﻨﺖ ﻣﺘﺮﺩﺩ ﺗﻨﺰﻳﻞ ﺍﻟﺘﻄﺒﻴﻖ ﻛﻨﺖ ...,ﻟﻔﺘﺮﺓ ﻛﺒﻴﺮﺓ ﺍﻟﺰﻣﻦ ﻛﻨﺖ ﻣﺘﺮﺩﺩ ﺗﻨﺰﻳﻞ ﺍﻟﺘﻄﺒﻴﻖ ﻛﻨﺖ ...


In [27]:
pos_content2 = []
for i in range(len(train_revSA)):

    arabic_text = train_revSA["content_lemmatized"][i]

    # Tokenize the sentence
    tokens = word_tokenize(arabic_text)

    pos_tags = nltk.pos_tag(tokens)
    # pos_tags = [nltk.pos_tag(e) for e in tokens]

    pos_content2.append(pos_tags)

    if i == 1:
        # print("arabic_text : ", tokens)
        print("NLTK-based POS tagging: ", tagged_words)

NLTK-based POS tagging:  [('ﺍﺭﺳﻞ', 'اسم'), ('ﺻﺪﻳﻖ', 'اسم'), ('ﺭﺍﺑﻂ', 'اسم'), ('ﻟﻌﺒﺔ', 'اسم'), ('ﻭﻗﺪ', 'اسم'), ('ﺭﺑﺤﺖ', 'اسم'), ('ﻣﻨﺘﺠﺎﺕ', 'اسم'), ('ﻣﺠﺎﻧﻴﺔ', 'اسم'), ('ﺍﻟﺘﻄﺒﻴﻖ', 'اسم'), ('ﺑﺪﺃ', 'اسم'), ('ﻳﻄﻠﺐ', 'اسم'), ('ﻣﻨﻲ', 'اسم'), ('ﺍﺷﻴﺎﺀ', 'اسم'), ('ﻛﺜﻴﺮﺓ', 'اسم'), ('ﻭﻓﻲ', 'اسم'), ('ﻣﺮﺓ', 'اسم'), ('ﺍﻧﻔﺬ', 'اسم'), ('ﻃﻠﺒﻪ', 'اسم'), ('ﻳﺨﺼﻢ', 'اسم'), ('ﺟﺰﺀ', 'اسم'), ('ﻣﺒﻠﻎ', 'اسم'), ('ﺍﻟﻔﺎﺗﻮﺭﺓ', 'اسم'), ('ﻭﻓﻲ', 'اسم'), ('ﺍﺧﺮ', 'اسم'), ('ﺍﻟﻤﻄﺎﻑ', 'اسم'), ('ﺍﺣﺼﻞ', 'اسم'), ('ﺍﻟﻬﺪﻳﺔ', 'اسم'), ('ﻻﻧﻪ', 'اسم'), ('ﺍﺻﺒﺢ', 'اسم'), ('ﺩﻋﺎﻳﺔ', 'اسم'), ('ﻳﺨﺼﻢ', 'اسم'), ('ﻫﻠﻼﺕ', 'اسم'), ('ﻓﻘﻂ', 'اسم'), ('ﻟﺬﻟﻚ', 'اسم'), ('ﺍﻟﺘﻄﺒﻴﻖ', 'اسم'), ('ﻳﻌﺘﺒﺮ', 'اسم'), ('ﻣﺨﺎﺩﻉ', 'اسم'), ('ﻭﺳﻴﺊ', 'اسم'), ('ﺟﺪﺍ', 'اسم'), ('ﺍﻧﺼﺢ', 'اسم'), ('ﺑﺘﻨﺰﻳﻠﻪ', 'اسم')]


In [35]:
from snowballstemmer import stemmer

ar_stemmer = stemmer("arabic")
ar_stemmer.stemWord(u'سيراقبونه')


'يراقب'

In [55]:
from tashaphyne.stemming import ArabicLightStemmer

ArListem = ArabicLightStemmer()

word = u'سيراقبونه'

ArListem.light_stem(word)

#extract stem
print(ArListem.get_stem())

#extract root
print(ArListem.get_root())

راقب
رقب


In [38]:
#extract stem
print(ArListem.get_stem())

#extract root
print(ArListem.get_root())

راقب
رقب


In [39]:
print(ArListem.get_prefix())

سي


In [42]:
print(ArListem.get_prefix(1))

س


In [43]:
print(ArListem.get_suffix())

ونه


In [44]:
print(ArListem.get_affix())

سي-ونه
